# Lib Install

In [24]:
!pip install langchain faiss-cpu langchain_community sentence-transformers langchain-groq langchainhub

# Import e Defines

In [182]:
import json
import pickle
import numpy as np
import pandas as pd
from getpass import getpass
from typing import List

import faiss

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


def edit_docs(docs):
    return f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)])

def pretty_print_docs(docs):
    print(edit_docs(docs))

In [169]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


# Load CCB Data

In [5]:
with open("codigo_civil.json", "r") as f:
    cod_civil = json.load(f)

In [10]:
def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

result = collect_artigo_and_header(cod_civil)
documents = result['articles']

In [11]:
len(documents)

318

In [12]:
pd.Series([len(doc) for doc in documents]).describe()

count    318.000000
mean       6.468553
std        4.647575
min        1.000000
25%        3.000000
50%        5.000000
75%        9.000000
max       24.000000
dtype: float64

# Load Embeddings

In [13]:
with open("/content/codigocivil_embeddings_object.pkl", 'rb') as f:
    all_embeddings = pickle.load(f)

embeddings = [group['mean_embedding'] for group in all_embeddings]

In [14]:
embeddings = np.vstack(embeddings)
embeddings.shape

(318, 1024)

# Load LLM

In [170]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.019196442, 'prompt_time': 0.005417684, 'queue_time': None, 'total_time': 0.024614126}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c382efc-c81e-421a-a538-7602208e87f7-0')

# Construindo o Retriver com os embeddings pré construidos

## Carregando modelo que gera os embeddings da Query (mesmo que gerou os embeddings dos documentos)

In [16]:
# Carregar o modelo de embeddings da Hugging Face
hf_embeddings = HuggingFaceEmbeddings(model_name="rufimelo/Legal-BERTimbau-sts-large-ma-v3")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


## Converte os documentos em Indices no formato do LangChain e Cria o Vector DATABASE com FAISS

Isso facilita o processo de busca e de comparação entre os embeddings
o IndexFlatL2 e o default do FAISS é distancia Euclidiana. (Alternativa estudar uma forma de fazer similaridade de cossenos)


In [17]:
# Criar uma lista de objetos Document
docs = [Document(page_content='\n'.join(article_group)) for article_group in documents]
# Criar um InMemoryDocstore
docstore = InMemoryDocstore(dict(enumerate(docs)))

# Mapear índices para documentos
index_to_docstore_id = {i: i for i in range(len(docs))}

# Criar um índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension) # Euclidian distance
index.add(embeddings)

# Criar o vetor FAISS a partir dos embeddings carregados e incluir a função de embeddings
vectorstore = FAISS(
    embedding_function=hf_embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# Configurar o retriever para buscar os documentos relevantes
retriever = vectorstore.as_retriever()

## Testando o Retriver

In [20]:
pretty_print_docs(retriever.invoke('O que é Casamento?'))

Document 1:

Art. 1.723. É reconhecida como entidade familiar a união estável entre o homem e a mulher, configurada na convivência pública, contínua e duradoura e estabelecida com o objetivo de constituição de família.

§ 1 o A união estável não se constituirá se ocorrerem os impedimentos do art. 1.521 ; não se aplicando a incidência do inciso VI no caso de a pessoa casada se achar separada de fato ou judicialmente.

§ 2 o As causas suspensivas do art. 1.523 não impedirão a caracterização da união estável.

  
Art. 1.724. As relações pessoais entre os companheiros obedecerão aos deveres de lealdade, respeito e assistência, e de guarda, sustento e educação dos filhos.

  
Art. 1.725. Na união estável, salvo contrato escrito entre os companheiros, aplica-se às relações patrimoniais, no que couber, o regime da comunhão parcial de bens.

  
Art. 1.726. A união estável poderá converter-se em casamento, mediante pedido dos companheiros ao juiz e assento no Registro Civil.

  
Art. 1.727. As 

## Testando Retriver Compresion (Sumarização)

In [21]:
compressor_llm = ChatGroq(temperature=0.5, model_name="llama3-70b-8192", api_key=GROQ_API_KEY)
compressor = LLMChainExtractor.from_llm(compressor_llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "O que é Casamento?"
)
pretty_print_docs(compressed_docs)

Document 1:

Art. 1.726. A união estável poderá converter-se em casamento, mediante pedido dos companheiros ao juiz e assento no Registro Civil.
----------------------------------------------------------------------------------------------------
Document 2:

Art. 1.511. O casamento estabelece comunhão plena de vida, com base na igualdade de direitos e deveres dos cônjuges.

Art. 1.514. O casamento se realiza no momento em que o homem e a mulher manifestam, perante o juiz, a sua vontade de estabelecer vínculo conjugal, e o juiz os declara casados.


# Criando Agente baseado no Prompt do React

## Definindo Tool de Retriver

In [205]:
retriever_tool = create_retriever_tool(
    compression_retriever, # retriever
    "busca_codigo_civil",
    "Busca de artigos no Código Civil Brasileiro. Use para encontrar os artigos que abordam um determinado assunto ou para obter o texto de um artigo.",
    document_separator =''
)

tools = [retriever_tool]

## Obtenção de prompt modelo do Prompt


In [206]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
template = prompt.template
print(template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [207]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


## Possíveis Prompts

https://github.com/joaomdmoura/crewAI/issues/103#issuecomment-1908792048

These keywords must never be translated and transformed:
- Action:
- Thought:
- Action Input:
because they are part of the thinking process instead of the output.

In [208]:
custom_template = """
Você é um agente que tem acesso ao Código Civil Brasileiro por meio de uma ferramenta de busca.
Sua tarefa é encontrar os artigos relevantes para uma pergunta de entrada.
Além de citar os artigos, destaque os trechos dos artigos que melhor explicam a pergunta.

NÃO SE PREOCUPE EM RESPONDER A PERGUNTA.

Encontre seguintes perguntas da melhor forma possível. Você tem acesso às seguintes ferramentas:

{tools}

Use o seguinte formato:

Question: a pergunta de entrada que você deve encontrar os artigos relevantes
Thought: você deve sempre pensar sobre o que fazer
Action: a ação a ser tomada, deve ser uma das seguintes [{tool_names}]
Action Input: a entrada para a ação
Observation: o resultado da ação
... (este Thought/Action/Action Input/Observation pode se repetir N vezes)
Thought: Eu sei todos os artigos necessários para responder a pergunta
Final Answer: lista de artigos e lista de trechos que ajudam a responder a pergunta de entrada original

Após gerar a saída final encerre sua atividade.

Comece!

Pergunta: {input}
Pensamento:{agent_scratchpad}

Instruções de formato Resposta Final: {format_instructions}
""".strip()

In [228]:
custom_template_oneshot ="""
Você é um agente que tem acesso ao Código Civil Brasileiro por meio de uma ferramenta de busca.
Sua tarefa é encontrar os artigos relevantes para uma pergunta de entrada.
Além de citar os artigos, destaque os trechos dos artigos que melhor explicam a pergunta.

NÃO SE PREOCUPE EM RESPONDER A PERGUNTA.

Quebre a pergunta nas sentenças (I, II, ...) ou alternativas (A, B, C, D) presentes nela. Veja o exemplo
# Exemplo de pergunta:
A sucessão testamentária é um componente [...] Em relação à sucessão testamentária, analise as afirmativas a seguir.
I. Ao cego só se permite o testamento público [..] fazendo-se de tudo circunstanciada menção no testamento.
II. O testamento cerrado deve ser escrito [...] com a sua assinatura, todas as páginas.
III. Em circunstâncias excepcionais [...] a critério do juiz.
IV. O testamento particular pode ser [...] que as testemunhas a compreendam.
Está correto o que se afirma apenas em
A) I e II.
B) I e IV.
C) II e IV.
D) III e IV.
# Exemplo de quebra de pergunta:
Quais artigos se relacionam com a sucessão tesmentária e o tópico I?
[...]
Quais artigos se relacionam com a sucessão tesmentária e o tópico IV?
# Exemplo de pensamento unindo as partes:
I. É verdadeiro em relação ao Art. 1.867, pois [...]
II - É falso pois o Art. 1.868 diz que [...]
III - Falso. De acordo com o Art. 1.879 em circunstâncias [...]
IV - Verdadeiro, conforme Art. 1.880.
# Exemplo de resposta final esperada:
Art. 1.867, Art. 1.867, Art. 1.879, Art. 1.880.

Encontre seguintes perguntas da melhor forma possível. Você tem acesso às seguintes ferramentas:

{tools}

Use o seguinte formato:

Question: a pergunta de entrada que você deve encontrar os artigos relevantes
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: lista de artigos e lista de trechos que ajudam a responder a pergunta de entrada original

Após gerar a saída final encerre sua atividade.

Comece!

Pergunta: {input}
Pensamento:{agent_scratchpad}

Instruções de formato Resposta Final: {format_instructions}
""".strip()

In [241]:
custom_template_comment = """
Você é um agente que tem acesso ao Código Civil Brasileiro por meio de uma ferramenta de busca.
Sua tarefa é encontrar os artigos relevantes para uma pergunta de entrada.
Além de citar os artigos, destaque os trechos dos artigos que melhor explicam a pergunta.

NÃO SE PREOCUPE EM RESPONDER A PERGUNTA.

Elabore um pensamento sobre cada alternativa (A, B, C, D, ...) ou afirmação (I, II, III, ...) presente na pergunta.

Para te ajudar irei fornecer comentários de alunos de direito sobre a pergunta. Esses comentários
podem estar corretos ou não, o importante é observar os artigos citados.

Encontre seguintes perguntas da melhor forma possível. Você tem acesso às seguintes ferramentas:

{tools}

Use o seguinte formato:

Question: a pergunta de entrada que você deve encontrar os artigos relevantes
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: lista de artigos e lista de trechos que ajudam a responder a pergunta de entrada original

Não gere notas adicionais na saída final. Comece!

Pergunta: {input}
Comentários: {comentarios}
Pensamento:{agent_scratchpad}

Instruções de formato Resposta Final: {format_instructions}
"""

## Organizando Prompt

In [242]:
class GET_CONTEXT_JSON(BaseModel):
    artigos_relevantes: List[str] = Field(description="artigos relevantes para a pergunta")
    trechos_artigos: List[str] = Field(description="trechos dos artigos relevantes para a pergunta")
    notas: str = Field(description="notas sobre os artigos escolhidos")
    resposta_provavel: str = Field(description="alternativa da resposta provavel para a pergunta")
    justificativa_resposta: str = Field(description="justificativa da resposta provavel")
output_parser = JsonOutputParser(pydantic_object=GET_CONTEXT_JSON)

In [243]:
template_to_use = custom_template_comment

prompt.template = template_to_use.strip()
prompt.partial_variables = {"format_instructions": output_parser.get_format_instructions()}
if template_to_use == custom_template_comment:
    print("[Não faço parte do prompt] Observação: Comentários adicionado\n\n")
    prompt.input_variables.append('comentarios')

print(prompt.template)

[Não faço parte do prompt] Observação: Comentários adicionado


Você é um agente que tem acesso ao Código Civil Brasileiro por meio de uma ferramenta de busca.
Sua tarefa é encontrar os artigos relevantes para uma pergunta de entrada.
Além de citar os artigos, destaque os trechos dos artigos que melhor explicam a pergunta.

NÃO SE PREOCUPE EM RESPONDER A PERGUNTA.

Elabore um pensamento sobre cada alternativa (A, B, C, D, ...) ou afirmação (I, II, III, ...) presente na pergunta.

Para te ajudar irei fornecer comentários de alunos de direito sobre a pergunta. Esses comentários
podem estar corretos ou não, o importante é observar os artigos citados.

Encontre seguintes perguntas da melhor forma possível. Você tem acesso às seguintes ferramentas:

{tools}

Use o seguinte formato:

Question: a pergunta de entrada que você deve encontrar os artigos relevantes
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input

## Definindo Agente (React Based)

In [247]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=10) #handle_parsing_errors=True

## Testando

In [248]:
pergunta_teste = """Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego. Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego. Diante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis. B) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção. C) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis. D) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção. E) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis."""
comentario_teste = """CC
Art. 1.826. O possuidor da herança está obrigado à restituição dos bens do acervo, fixando-se-lhe a responsabilidade segundo a sua posse, observado o disposto nos arts. 1.214 a 1.222.
Parágrafo único. A partir da citação, a responsabilidade do possuidor se há de aferir pelas regras concernentes à posse de má-fé e à mora.
Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.
Parágrafo único. Os frutos pendentes ao tempo em que cessar a boa-fé devem ser restituídos, depois de deduzidas as despesas da produção e custeio; devem ser também restituídos os frutos colhidos com antecipação.
Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.
Art. 1.973. Sobrevindo descendente sucessível ao testador, que não o tinha ou não o conhecia quando testou, rompe-se o testamento em todas as suas disposições, se esse descendente sobreviver ao testador.
Art. 1.974. Rompe-se também o testamento feito na ignorância de existirem outros herdeiros necessários.
STF já decidiu que o rompimento do testamento só ocorre se provado que o testador não tinha conhecimento do novo herdeiro necessário.
Art. 1.975. Não se rompe o testamento, se o testador dispuser da sua metade, não contemplando os herdeiros necessários de cuja existência saiba, ou quando os exclua dessa parte.
Obs: Nesse caso, o testamento é modificado quanto à reservada legítima. (Fonte: https://www.jusbrasil.com.br/artigos/testamentos-invalidados-e-ineficazes-revogacao-rompimento-caducidade-anulabilidade-e-nulidade/759470990)
Na hipótese de nulidade de inventário, hipótese da questão, o prazo prescricional é de 10 (dez) anos, salvo melhor juízo. Isso porque, por dedução, nulidade de inventário não encontra previsão nas hipóteses do art. 206. Logo:
Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.
O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como os frutos já percebidos.""".strip()

In [249]:
if template_to_use == custom_template_comment:
    print('Com Comentarios')
    agent_executor.invoke({"input": pergunta_teste, "comentarios":comentario_teste})
else:
    agent_executor.invoke({"input": pergunta_teste})

Com Comentarios


> Entering new AgentExecutor chain...
Here is the response:

Question: Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego. Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego. Diante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a A) todos os frutos já percebidos, be

KeyboardInterrupt: 